In [1]:
import os
sep_local = os.path.sep
sep_local

'/'

In [2]:
%env TF_KERAS = 1

env: TF_KERAS=1


In [3]:
import sys
sys.path.append('..' + sep_local + '..' + sep_local + '..') # For Windows import

In [4]:
import os
os.chdir('..' + sep_local + '..' + sep_local + '..') # For Linux import


In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

/home/azeghost/git/new_GAN_models/Generative_Models


In [6]:
#IMG_DIR = 'D:\\datasets\\Ms_Pacman\\Ms_Pacman\\Test\\'
IMG_DIR = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux

Using TensorFlow backend.


In [7]:
#IMG_DIR = '..\..\..\\data\\atari_v1\\screens\\mspacman'
VAL_PCT = 25
VAL_FORMAT = 'png'

In [8]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

## downscalse the image into fifth

In [ ]:
scale=5
IMG_SIZE=(160//scale, 210//scale, 3)
BATCH_SIZE = 5
EPIS_LEN = 10
EPIS_SHIFT = 5

In [10]:
#we created new  class_mode  episode_flat witch returns 
#da.from_array(np.reshape(batch_x, (-1,)+self.image_shape )), da.from_array(np.reshape(batch_gt, (-1,)+self.image_shape))

In [11]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [12]:
import tensorflow as tf
import numpy as np

  INFO     | Found 1177538 training files


  INFO     | Found 383441 validation files


In [13]:
#changed output shapes of Dataset Generators 
#from (BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE)
#to (BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE) 

In [14]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)


In [15]:
dataset_name = 'atari_v1_Pacman'

In [16]:
intermediate_dim = 10
inputs_shape= IMG_SIZE # image shape
LATENT_DIM = 10

In [17]:
inputs_shape

In [18]:
#Not used here

enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(LATENT_DIM)
]

dec_lays2 = [
    tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME"),

]

(32, 42, 3)

In [19]:

enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

In [20]:
model_name = 'Atari_AE_Dense'

In [21]:
import os
#experiments_dir='..'+sep_local+'..'+sep_local+'..'+sep_local+'experiments'+sep_local + model_name
experiments_dir=os.getcwd()+ sep_local  +'experiments'+sep_local + model_name

In [22]:
from os.path import abspath
absolute = abspath(experiments_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())


'../../../experiments/Atari_AE/Atari_AE'

In [ ]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':LATENT_DIM,
        'layers': enc_mean_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':LATENT_DIM,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

In [ ]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=LATENT_DIM,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None #to restore trained model, set filepath=_restore
    )

In [ ]:
ae.compile(metrics=None)

In [ ]:
train_ds

In [ ]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver


In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [ ]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latent_shape=LATENT_DIM, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [ ]:
ae.fit(
    x=train_ds,
    steps_per_epoch=1000,
    epochs=100, 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

In [23]:
ae.fit(
    x=train_ds,
    steps_per_epoch=1000,
    epochs=100, 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

In [24]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':LATENT_DIM,
        'layers': enc_mean_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':LATENT_DIM,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [25]:
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(experiments_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)

In [26]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=LATENT_DIM,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None #to restore trained model, set filepath=_restore
    )

'../../../experiments/Atari_AE/var_save_dir'

In [27]:
ae.compile(metrics=None)

In [28]:
train_ds

In [29]:
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver


'../../../experiments/Atari_AE/var_save_dir'

  WARNING  | None


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32, 42, 10)        40        
_________________________________________________________________
dense_1 (Dense)              (None, 32, 42, 10)        110       
_________________________________________________________________
dense_2 (Dense)              (None, 32, 42, 10)        110       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 42, 10)        40        
_________________________________________________________________
dropout (Dropout)            (None, 32, 42, 10)        0         
_________________________________________________________________
activity_regularization (Act (None, 32, 42, 10)        0         
_________________________________________________________________
inference_outputs (Activatio (None, 32, 42, 10)        0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_7 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_8 (Dense)              (None, 20)                420       
_________________________________________________________________
batch_normalization_1 (Batch (None, 20)                80        
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
activity_regularization_1 (A (None, 20)                0         
_________________________________________________________________
generative_outputs (Activati (None, 20)                0

ValueError: Input 0 of layer batch_normalization_1 is incompatible with the layer: expected ndim=2, found ndim=4. Full shape received: [None, 32, 42, 20]

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [ ]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latent_shape=LATENT_DIM, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [ ]:
ae.fit(
    x=train_ds,
    steps_per_epoch=1000,
    epochs=100, 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)